# No Provision

In this strategy, half of the portfolio is in token zero, and the other half is in token one. The assets will not be deposited in any pool. This simple strategy is a good benchmark because it shows how much the value of the tokens increased. Since no liquidity was provided, there is no impermanent loss of earned fees. This strategy will be our reference strategy, which we will take as a baseline.

In [15]:
from datetime import date, datetime
from _decimal import Decimal
from typing import List, Dict
import pandas as pd

from demeter import TokenInfo, Actuator, Strategy, RowData, ChainType, MarketInfo, AtTimeTrigger, EvaluatorEnum
from demeter.uniswap import UniV3Pool, UniLpMarket

In [16]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

In [17]:
class NoProvisionStrategy(Strategy):
    def initialize(self):
        """
        Initialize function will be called right before a backtest start.
        You can do various things here, e.g. register a trigger, or add a simple moving average line.
        """
        new_trigger = AtTimeTrigger(  # define a new trigger
            time=datetime(2022, 8, 20, 12, 0, 0), do=self.work  # we will make the action happen at 12:00 20/8/22
        )  # This is a callback function, defines what to do at this time.
        self.triggers.append(new_trigger)  # Register our trigger

    def work(self, row_data: RowData):
        """
        When time is up, work function will be called.
        """
        pass

In [18]:
# Declare a token, and it's name will be used as unit of amounts.
usdc = TokenInfo(name="usdc", decimal=6)  # declare token usdc
eth = TokenInfo(name="eth", decimal=18)  # declare token eth
# Declare an Uniswap V3 pool. We will set the parameters according to the real pool on chain.
pool = UniV3Pool(token0=usdc, token1=eth, fee=0.05, base_token=usdc)

# Declare a market key, which will be used to find the corresponding market in broker
market_key = MarketInfo("U2EthPool")
# Declare the market,
market = UniLpMarket(market_key, pool)  # uni_market:UniLpMarket, positions: 0, total liquidity: 0
# load data for market. those data is prepared by download tool
market.data_path = "../data"  # set data path
market.load_data(
    chain=ChainType.polygon.name,  # load data
    contract_addr="0x45dda9cb7c25131df268515131f647d726f50608",
    start_date=date(2023, 8, 15),
    end_date=date(2023, 8, 15),
)

# Declare the Actuator, which controls the whole process
actuator = Actuator()  # declare actuator, Demeter Actuator (broker:assets: ; markets: )
# add market to broker
actuator.broker.add_market(market)
# Initial some fund to broker.
actuator.broker.set_balance(usdc, 10000)
actuator.broker.set_balance(eth, 10)
# Set strategy to actuator
actuator.strategy = NoProvisionStrategy()  # set strategy to actuator
# Set price. Those price will be used in all markets.
# Usually, you will have to find the price list from outer source.
# Luckily, uniswap pool data contains price information. So UniLpMarket provides a function to retrieve price list.
actuator.set_price(market.get_price_from_data())
# run test, If you use default parameter, final fund status will be printed in console.

# actuator.run()

2023-11-02 17:14:06,881 - INFO - start load files from 2023-08-15 to 2023-08-15...
2023-11-02 17:14:06,893 - INFO - load file complete, preparing...
2023-11-02 17:14:06,974 - INFO - data has been prepared


In [19]:
actuator.run(evaluator=[
        EvaluatorEnum.max_draw_down,
        EvaluatorEnum.annualized_returns,
        EvaluatorEnum.net_value,
        EvaluatorEnum.profit,
        EvaluatorEnum.net_value_up_down_rate,
        EvaluatorEnum.eth_up_down_rate,
    ])
evaluating_result: Dict[EvaluatorEnum, Decimal] = actuator.evaluating_indicator

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2023-11-02 17:14:06,992 - INFO - init strategy...
2023-11-02 17:14:06,994 - INFO - start main loop...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1440/1440 [00:00<00:00, 1662.78it/s]
2023-11-02 17:14:07,866 - INFO - main loop finished
2023-11-02 17:14:07,878 - INFO - Start calculate evaluating indicator...
2023-11-02 17:14:07,881 - INFO - Evaluating indicator has finished it's job.
2023-11-02 17:14:07,882 - INFO - Print actuator summary
2023-11-02 17:14:07,896 - INFO - Backtesting finished, execute time 0.906944751739502s
2023-11-02 17:14:07,918 - INFO - files have saved to ./result/backtest-20231102-171407.account.csv,./result/backtest-20231102-171407.action.json,./result/backtest-20231102-171407.action.pkl


Broker                                            
Asset amounts                 
USDC      :10000                    ETH       :10                       
Markets                       
U2EthPool(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee       :0.0500                   is 0 base :True                     
positions 
Empty DataFrame


Account Status                                    
                                         net_value   USDC ETH U2EthPool_net_value U2EthPool_base_uncollected U2EthPool_quote_uncollected U2EthPool_base_in_position U2EthPool_quote_in_position  U2EthPool_position_count
2023-08-15 00:00:00  28436.94418788503400887972931  10000  10               0E-24                          0                           0                          0                           0                         0
2023-08-15 00:01:00  28436.94418788503400887972931  10000  10               0E-24                          0                      

['./result/backtest-20231102-171407.account.csv',
 './result/backtest-20231102-171407.action.json',
 './result/backtest-20231102-171407.action.pkl']